In [1]:
from collections import Counter
import csv
import pandas as pd
from IPython.core.debugger import Tracer; debug_here = Tracer()

annotate = set()

In [10]:
# Talk page usernames
talk_csvpath = '/home/michael/school/research/wp/wikipedia/data/talk/ipc_utf8_talkpages.csv'
talk_counter = Counter()

with open(talk_csvpath, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        talk_counter[row[3]] += 1

# print(talk_counter.most_common())
print(len(talk_counter))

5537


In [3]:
# ARTICLE USERNAMES
art_filepath = '/home/michael/school/research/wp/wikipedia/data/ipc_article_biases.csv'

article_usernames = []

with open(art_filepath, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        article_usernames.append(row[2])

article_counter = Counter(article_usernames)
len(article_counter)

199370

In [4]:
# Editors in both article and talk pages
both_usernames = []

for name in article_counter.keys():
    if name in talk_counter:
        both_usernames.append(name)
        
len(both_usernames)

4371

In [5]:
# Filter by number of talk page contributions
thresholds = range(3,16)
talk_eds = {}

for t in thresholds:
    talk_eds[t] = [ed for ed in talk_counter if talk_counter[ed] >= t]
    print(t, len(talk_eds[t]))

3 2050
4 1537
5 1225
6 994
7 864
8 776
9 689
10 614
11 546
12 506
13 470
14 449
15 410


In [15]:
threshold = 5
annotate = set(talk_eds[threshold]).intersection(both_usernames)
len(annotate)

1150

In [16]:
# Filter by editors with citations
citation_csvpath = '/home/michael/school/research/wp/wikipedia/data/editor_citation_matrix.csv'
ed_cite = pd.read_csv(citation_csvpath, index_col=0)
ed_cite

ed_cites = Counter() 
for name, row in ed_cite.iterrows():
    ed_cites[name] = sum(row.values)
    
annotate = set(annotate).intersection(set(ed for ed in ed_cites if ed_cites[ed] >= 3))
len(annotate)

381

In [17]:
# Filter by editors who appear in at least one thread with other editors in annotate
talk_data = pd.read_csv(talk_csvpath)
eds_in_conversation = set()
prev_thread = ''
thread_eds = set()
for i,row in talk_data.iterrows():
    if row['thread_title'] == prev_thread:
        if row['username'] in annotate:
            thread_eds.add(row['username'])
    else:
        if len(thread_eds) > 1:
            eds_in_conversation |= thread_eds
        thread_eds = set()
    prev_thread = row['thread_title']

len(eds_in_conversation)

358